In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math 
import geohash2
import torch.nn.functional as F

# 필요 데이터 불러오기

In [2]:
def read_data(file_path):
    data = pd.read_csv(file_path)
    return data

In [3]:
# 1. 전체 데이터
nyc_df = pd.read_csv('../data/nyc/raw/NYC_df.csv')
tky_df = pd.read_csv('../data/tky/raw/TKY_df.csv')

# POI 그래프 
nyc_poi_traj_graph = pd.read_csv('../data/nyc/graph/nyc_traj_graph.csv')
nyc_poi_space_graph = pd.read_csv('../data/nyc/graph/nyc_space_graph_02km.csv')
nyc_poi_time_graph = pd.read_csv('../data/nyc/graph/nyc_time_graph_09.csv')

tky_poi_traj_graph = pd.read_csv('../data/tky/graph/tky_traj_graph.csv')
tky_poi_space_graph = pd.read_csv('../data/tky/graph/tky_space_graph_01km.csv')
tky_poi_time_graph = pd.read_csv('../data/tky/graph/tky_time_graph_09.csv')

# Space GAT 임베딩
nyc_geohash_emb = pd.read_csv('../data/nyc/graph/nyc_geohash_gat_space_embedding.csv')
tky_geohash_emb = pd.read_csv('../data/tky/graph/tky_geohash_gat_space_embedding.csv')

In [4]:
nyc_poi_time_graph

,src,dst,weight
0,3fd66200f964a52000e71ee3,42699000f964a52047211fe3,0.942901
1,3fd66200f964a52000e71ee3,4a3b9ffdf964a520a8a01fe3,0.944565
2,3fd66200f964a52000e71ee3,4aaf803ef964a520376420e3,0.908647
3,3fd66200f964a52000e71ee3,4ad7392af964a520060921e3,0.918441
4,3fd66200f964a52000e71ee3,4ae4eb45f964a5206e9f21e3,0.910065
...,...,...,...
25647,509817afc84c2e276bc2cd1f,4ef0e7cf7beb5932d5bdeb4e,0.903352
25648,509aaca2fe70fdaaea894fd3,424de080f964a520b6201fe3,0.950112
25649,509aaca2fe70fdaaea894fd3,4ad9267ef964a5209b1821e3,0.926381
25650,509aaca2fe70fdaaea894fd3,4e8a2aa061aff1e909c59333,0.914518


# 모델 구조 생성

In [5]:
nyc_df.head(2)

,UserId,PoiId,PoiCategoryId,PoiCategoryName,Latitude,Longitude,TimezoneOffset,UTCTime,LocalTime,Weekday,Hour,Day,UserRank,SplitTag,Holiday,UpperCategory,TrajectoryId
0,1,4db44994cda1c57c82583709,4bf58dd8d48988d1f1931735,General Entertainment,40.739398,-73.993210,-240,2012-04-08 18:20:29,2012-04-08 14:20:29,6,14,2012-04-08,1.0,train,True,Culture & Leisure,1_1
1,1,40f1d480f964a5205b0a1fe3,4bf58dd8d48988d143941735,Breakfast Spot,40.719929,-74.008532,-240,2012-04-09 16:20:52,2012-04-09 12:20:52,0,12,2012-04-09,2.0,train,False,Commercial/Services,1_1


In [6]:
# 데이터 분할
nyc_train = nyc_df[nyc_df['SplitTag'] == 'Train']
nyc_val = nyc_df[nyc_df['SplitTag'] == 'Val']
nyc_test = nyc_df[nyc_df['SplitTag'] == 'Test']

tky_train = tky_df[tky_df['SplitTag'] == 'Train']
tky_val = tky_df[tky_df['SplitTag'] == 'Val']
tky_test = tky_df[tky_df['SplitTag'] == 'Test']

# 초기 임베딩 생성

## (1) User 임베딩

In [ ]:
# --- 단기 선호도 ---
class UserShortPrefMemory(nn.Module):
    """
    Inference 전용 전역 단기메모리 버퍼.
    - 학습 시에는 build_functional_short_pref()로 (B,L,d) 단기 임베딩을 '계산'만 해서 쓰고,
      momentum_update()는 호출하지 않는 것을 권장합니다.
    """
    def __init__(self, num_users: int, dim: int, default_beta: float = 0.5):
        super().__init__()
        self.register_buffer('memory', torch.zeros(num_users, dim))  # (U, d)
        self.default_beta = float(default_beta)

    def forward(self, user_ids) -> torch.Tensor:
        # int, 0-d 텐서, (B,) 모두 지원
        if isinstance(user_ids, int):
            user_ids = torch.tensor([user_ids], device=self.memory.device, dtype=torch.long)
            return self.memory[user_ids][0]  # (d,)
        elif isinstance(user_ids, torch.Tensor) and user_ids.ndim == 0:
            return self.memory[user_ids.view(1)][0]  # (d,)
        else:
            user_ids = user_ids.to(device=self.memory.device, dtype=torch.long)
            return self.memory[user_ids]  # (B, d)

    @torch.no_grad()
    def momentum_update(self,
                        user_ids,
                        new_short_prefs: torch.Tensor,
                        beta: float = None):
        """
        Inference 전용: 전역 버퍼를 모멘텀으로 갱신.
        user_ids: int | 0-d tensor | (B,) LongTensor
        new_short_prefs: (d,) or (B, d)
        """
        b = float(self.default_beta if beta is None else beta)
        b = max(0.0, min(1.0, b))  # clamp

        # 표준화
        if isinstance(user_ids, int):
            user_ids = torch.tensor([user_ids], device=self.memory.device, dtype=torch.long)
        elif isinstance(user_ids, torch.Tensor) and user_ids.ndim == 0:
            user_ids = user_ids.view(1).to(self.memory.device, torch.long)
        else:
            user_ids = user_ids.to(self.memory.device, torch.long)

        v = new_short_prefs.to(self.memory.device, self.memory.dtype)
        if v.ndim == 1:
            v = v.unsqueeze(0)  # (1, d)

        # (B,)와 (B,d) 길이 일치 확인
        if user_ids.numel() != v.size(0):
            # 단일 벡터로 여러 사용자에 동일 업데이트를 의도한 경우 허용
            if v.size(0) == 1:
                v = v.expand(user_ids.numel(), -1)
            else:
                raise ValueError(f"Shape mismatch: user_ids={user_ids.shape}, new_short_prefs={new_short_prefs.shape}")

        # 동일 user_id가 B 내에 중복될 수 있으면 마지막 항으로 덮어쓰기(간단/안전)
        self.memory[user_ids] = b * self.memory[user_ids] + (1.0 - b) * v

    @torch.no_grad()
    def reset_users(self, user_ids):
        if isinstance(user_ids, int):
            user_ids = torch.tensor([user_ids], device=self.memory.device, dtype=torch.long)
        elif isinstance(user_ids, torch.Tensor) and user_ids.ndim == 0:
            user_ids = user_ids.view(1).to(self.memory.device, torch.long)
        else:
            user_ids = user_ids.to(self.memory.device, torch.long)
        self.memory.index_fill_(0, user_ids, 0.0)

    # ===== 학습용(배치) 함수형 단기 메모리 생성기 =====
    @staticmethod
    def build_functional_short_pref(poi_seq_emb: torch.Tensor,
                                    K: int = 2,
                                    beta: float = 0.5) -> torch.Tensor:
        """
        학습 시 사용: 전역 버퍼를 수정하지 않고, 과거 K 스텝을 EMA로 요약한 단기 임베딩 시퀀스를 생성.
        poi_seq_emb: (B, L, d)  # 각 시점의 POI 임베딩(예: E_poi[poi_ids])
        return:      (B, L, d)  # t 시점에서 과거 ≤K 스텝의 EMA (자기 자신 제외; t=0은 0 벡터)
        """
        B, L, d = poi_seq_emb.shape
        out = poi_seq_emb.new_zeros(B, L, d)
        if L <= 1:
            return out
        K = int(max(1, K))
        beta = float(max(0.0, min(1.0, beta)))

        for t in range(1, L):
            # window: (B, w, d), where w = min(K, t)
            w = min(K, t)
            window = poi_seq_emb[:, t - w:t, :]  # 과거 구간만
            # 순차 EMA
            v = window[:, 0, :]
            for k in range(1, w):
                v = beta * v + (1.0 - beta) * window[:, k, :]
            out[:, t, :] = v
        return out



# --- 장기 선호도(학습) ---
class UserLongPrefMemory(nn.Module):
    def __init__(self, num_users: int, dim: int):
        super().__init__()
        self.long_pref_emb = nn.Embedding(num_users, dim)    # 학습될 장기 선호도 

    def forward(self, user_ids) -> torch.Tensor:
        if isinstance(user_ids, int):
            user_ids = torch.tensor([user_ids], device=self.long_pref_emb.weight.device, dtype=torch.long)
            return self.long_pref_emb(user_ids)[0]      # (d,)
        elif isinstance(user_ids, torch.Tensor) and user_ids.ndim == 0:
            return self.long_pref_emb(user_ids.view(1))[0]  # (d,)
        return self.long_pref_emb(user_ids)             # (B, d)    -> 배치단위의 경우


# --- 결합 User Embedding ---
class UserEmbedding(nn.Module):
    """ Combine short-term and long-term user preferences into a single user embedding."""
    def __init__(self, num_users: int,  dim: int, dropout: float = 0.1,):
        super().__init__()
        self.short_pref_memory = UserShortPrefMemory(num_users, dim)   # 단기 선호도
        self.long_pref_memory  = UserLongPrefMemory(num_users, dim)    # 장기 선호도
        
        # 브랜치별 정규화 (스케일 정렬)
        self.ln_short = nn.LayerNorm(dim)
        self.ln_long  = nn.LayerNorm(dim)
        
        # 3) 결합 MLP + 정규화
        self.mlp = nn.Sequential(
            nn.Linear(2*dim, dim),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.ln_out = nn.LayerNorm(dim)   # 마지막 정규화
        
    def forward(self, user_ids):
        short_pref = self.short_pref_memory(user_ids)   # (d,) or (B, d)
        long_pref  = self.long_pref_memory(user_ids)    # (d,) or (B, d)
        
        s_pref = self.ln_short(short_pref)
        l_pref = self.ln_long(long_pref)

        combined = torch.cat([s_pref, l_pref], dim=-1)  # (2d,) or (B, 2d)
        user_emb = self.mlp(combined)                   # (d,) or (B, d)
        user_emb = self.ln_out(user_emb)
        return user_emb

    @torch.no_grad()
    def update_short_pref(self, user_ids, new_short_prefs: torch.Tensor, beta: float = 0.5):
        """
        Update the short-term preferences of users using momentum.
        Args:
            user_ids (Union[int, torch.Tensor]): The user IDs to update.
            new_short_prefs (torch.Tensor): The new short-term preferences.
            beta (float, optional): The momentum factor. Defaults to 0.5.
        """
        self.short_pref_memory.momentum_update(user_ids, new_short_prefs, beta)

In [8]:
# 단기 선호도 생성 확인
n_users = 5
dim = 4

# 초기화
mem = UserShortPrefMemory(n_users, dim)

# 전체 메모리 확인
print(mem.memory)

# 개별 유저 단기 선호 임베딩 확인
init_user0_short_pref = mem(torch.tensor(0))
print(init_user0_short_pref)

# 새로운 단기 선호 임베딩 업데이트
new_user0_short_pref = torch.tensor([0.1, 0.2, 0.3, 0.4])
mem.momentum_update(user_ids=0, new_short_prefs=new_user0_short_pref, beta=0.5)
print(mem(torch.tensor(0)))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([0., 0., 0., 0.])
tensor([0.0500, 0.1000, 0.1500, 0.2000])


## (2) POI 임베딩
-> 아마도 GPT2 기준 768 차원이 필요하니까 각 임베딩은 256차원으로 구성

### Time 임베딩

In [9]:
def _cyc_emb(x: torch.Tensor, period: int, K: int = 2) -> torch.Tensor:
    """
    주기(period)를 가진 정수/실수 시계열 x를 사인/코사인으로 임베딩.
    x: (...,)     // 예: weekday(0~6), hour(0~23), minute(0~59)
    return: 2K 차원 (..., 2K)
    """
    x = x.float()
    outs = []
    for k in range(1, K + 1):
        ang = 2 * math.pi * k * x / period
        outs.append(torch.sin(ang))
        outs.append(torch.cos(ang))
    return torch.stack(outs, dim=-1)  # (..., 2K)


class TimeEmbedding(nn.Module):
    """
    weekday, hour, holiday 정보를 시간 임베딩으로 변환
    """
    def __init__(self, dim: int, k_weekday: int=2, k_hour: int=2):
        super().__init__()
        self.kw = k_weekday
        self.kh = k_hour
        self.tdim = dim
        
        self.bdim = 2 * (k_weekday + k_hour) + 1  # holiday 포함 (기본 시간 특징 차원)
        
        # 나머지 임베딩과 차원 일치
        self.proj = nn.Sequential(
            nn.Linear(self.bdim, self.tdim),
            nn.ReLU(),
            nn.LayerNorm(self.tdim)
        )

    def forward(self, weekday: torch.Tensor, hour: torch.Tensor, holiday: torch.Tensor) -> torch.Tensor:
        """
        return: (..., self.tdim)
        (배치/시퀀스/단일 모두 OK: 입력 모양을 그대로 유지한 채 마지막 차원만 확장됩니다)
        """
        e_w = _cyc_emb(weekday, period=7,  K=self.kw)   # 체크인 요일 임베딩
        e_h = _cyc_emb(hour,    period=24, K=self.kh)   # 체크인 시간 임베딩

        # Holiday는 이진 스칼라 그대로 사용
        e_ho = holiday.float().unsqueeze(-1)            # (..., 1)
        
        x = torch.cat([e_w, e_h, e_ho], dim=-1)         # (..., bdim)

        # 최종 차원 조절
        x = self.proj(x)                                # (..., tdim)

        return x


In [10]:
# ✅ 1. 임베딩 설정
embedding_dim = 13  # 원하는 출력 차원 (예: 다른 임베딩과 맞추기 위해)
time_emb_module = TimeEmbedding(dim=embedding_dim, k_weekday=2, k_hour=2)

# ✅ 2. 테스트 입력 (batch 형태: (B,) or (B, L))
weekday = torch.tensor([0, 1, 2, 3])    # 월~목
hour = torch.tensor([0, 6, 12, 18])     # 0시, 6시, 12시, 18시
holiday = torch.tensor([0, 0, 1, 1])    # 평일/휴일 여부

# ✅ 3. 임베딩 계산
time_emb = time_emb_module(weekday, hour, holiday)

# ✅ 4. 출력 확인
print("입력 weekday:", weekday)
print("입력 hour:", hour)
print("입력 holiday:", holiday)
print("출력 임베딩 shape:", time_emb.shape)   # ➜ (4, 16)
print("출력 임베딩 예시:")
print(time_emb)

입력 weekday: tensor([0, 1, 2, 3])
입력 hour: tensor([ 0,  6, 12, 18])
입력 holiday: tensor([0, 0, 1, 1])
출력 임베딩 shape: torch.Size([4, 13])
출력 임베딩 예시:
tensor([[-0.3957,  1.7553, -0.8413,  0.3207,  0.9913, -0.2533, -0.8413, -0.8413,
          0.4678, -0.7508, -0.8413, -0.8413,  2.0711],
        [ 1.1460,  0.0782, -0.2919,  0.8377,  0.4162, -0.6473, -1.1300, -0.3626,
         -0.7430,  2.3664, -1.0816, -1.1300,  0.5418],
        [-0.6663,  1.8922, -0.6663, -0.6663, -0.6663,  0.4580, -0.6663, -0.6663,
          2.4529, -0.1563, -0.1608,  0.1784, -0.6663],
        [ 1.3309, -0.6050,  0.4458, -0.6050, -0.6050, -0.6050, -0.6050,  0.4914,
         -0.2215, -0.6050,  2.7933, -0.6050, -0.6050]],
       grad_fn=<NativeLayerNormBackward0>)


### POI 임베딩

In [ ]:
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data

# POI ID 맵핑
def build_poi_idmap(poi_df: pd.DataFrame, id_col="PoiId"):
    uniq = pd.Index(poi_df[id_col].unique())
    id2idx = {pid: i for i, pid in enumerate(uniq)}
    return id2idx, len(uniq)

# 그래프 tensor 변환
def edges_from_csv(df, id2idx, src_col="src", dst_col="dst", w_col="weight"):
    src = torch.tensor([id2idx[s] for s in df[src_col].tolist()], dtype=torch.long)
    dst = torch.tensor([id2idx[d] for d in df[dst_col].tolist()], dtype=torch.long)
    w   = torch.tensor(df[w_col].astype(float).to_numpy(), dtype=torch.float)
    edge_index = torch.stack([src, dst], dim=0)  # (2,E)
    return edge_index, w

# 그래프 데이터 객체 생성
def pyg_data(num_nodes, edge_index, edge_weight):
    return Data(num_nodes=num_nodes, edge_index=edge_index, edge_weight=edge_weight)

# 그래프 가중치 정규화
def normalize_edge_weight(edge_index, edge_weight, num_nodes):
    # D^{-1/2} A D^{-1/2} 스타일
    row, col = edge_index
    deg = torch.zeros(num_nodes, device=edge_index.device).scatter_add_(0, row, edge_weight)
    deg = deg.clamp_min(1e-12)
    norm = (deg[row].pow(-0.5) * edge_weight * deg[col].pow(-0.5))
    return norm

# 그래프 학습 GNN 모델 -> weight를 더 잘 반영하는 GCN VS GAT
class POIEncoderGCN(nn.Module):
    def __init__(self, in_dim: int, hid_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hid_dim)
        self.conv2 = GCNConv(hid_dim, out_dim)

    def forward(self, x: torch.Tensor, data) -> torch.Tensor:
        """
        x: 초기 POI 임베딩 (num_nodes, in_dim)
        data: PyG Data 객체 (edge_index, edge_weight 포함)
        return: 최종 POI 임베딩 (num_nodes, out_dim)
        """
        x = self.conv1(x, data.edge_index, data.edge_weight)
        x = torch.relu(x)
        x = self.conv2(x, data.edge_index, data.edge_weight)
        return x


class POIEncoderGAT(nn.Module):
    def __init__(self, in_dim: int, hid_dim: int, out_dim: int, heads: int = 4):
        super().__init__()
        self.conv1 = GATConv(in_dim, hid_dim, heads=heads, concat=True)
        self.conv2 = GATConv(hid_dim * heads, out_dim, heads=1, concat=False)

    def forward(self, x: torch.Tensor, data) -> torch.Tensor:
        """
        x: 초기 POI 임베딩 (num_nodes, in_dim)
        data: PyG Data 객체 (edge_index만 필요)
        return: 최종 POI 임베딩 (num_nodes, out_dim)
        """
        x = self.conv1(x, data.edge_index)  # attention 학습
        x = torch.relu(x)
        x = self.conv2(x, data.edge_index)
        return x


/home/hj/anaconda3/envs/nextpoi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# POI 임베딩
class POIEmbedding(nn.Module):
    def __init__(self, num_pois: int, dim: int):
        super().__init__()
        self.poi_emb = nn.Embedding(num_pois, dim)    # POI 임베딩

    def forward(self, poi_ids: torch.Tensor):
        poi_emb = self.poi_emb(poi_ids)
        return poi_emb

# 카테고리 임베딩
class CatEmbedding(nn.Module):
    def __init__(self, num_cats: int, dim: int):
        super().__init__()
        self.cat_emb = nn.Embedding(num_cats, dim)    # 카테고리 임베딩

    def forward(self, cat_ids: torch.Tensor):
        cat_emb = self.cat_emb(cat_ids)
        return cat_emb
    

# POI 임베딩 학습 파이프라인
class POIEmbeddingPipeline(nn.Module):
    def __init__(self, num_pois: int, num_cats: int, dim: int, hidden_dim: int, dropout=0.1, encoder_type: str = 'GCN'):
        super().__init__()
        self.poi_emb_module = POIEmbedding(num_pois, dim)    # POI 초기 임베딩 생성
        self.cat_emb_module = CatEmbedding(num_cats, dim)    # 카테고리 초기 임베딩 생성

        # POI 임베딩 인코딩 GNN 모델
        encoder_cls = POIEncoderGCN if encoder_type == 'GCN' else POIEncoderGAT
        
        # 각 그래프별 인코더 생성
        self.space_encoder = encoder_cls(in_dim=dim, hid_dim=hidden_dim, out_dim=dim)
        self.traj_encoder  = encoder_cls(in_dim=dim, hid_dim=hidden_dim, out_dim=dim)
        self.time_encoder  = encoder_cls(in_dim=dim, hid_dim=hidden_dim, out_dim=dim)
        
        # 최종 임베딩 결합
        total_dim = dim * 4    # POI, 카테고리, 공간, 이동, 시간 임베딩 결합
        self.merge = nn.Sequential(
            nn.Linear(total_dim, dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.LayerNorm(dim)
        )
        
    def forward(self, poi_ids, space_data, traj_data, time_data, cat_ids):
        poi_emb = self.poi_emb_module(poi_ids)   
        cat_emb = self.cat_emb_module(cat_ids)   
        
        # 3가지 그래프 인코더로 POI 임베딩 변환
        space_emb = self.space_encoder(poi_emb, space_data)
        traj_emb  = self.traj_encoder(poi_emb, traj_data)
        time_emb  = self.time_encoder(poi_emb, time_data)

        combined = torch.cat([cat_emb, space_emb, traj_emb, time_emb], dim=-1)
        final_emb = self.merge(combined)
        return final_emb     # 최종 POI 임베딩
        

### Space 임베딩

In [14]:
def get_space_embedding(lat: float, lon: float, space_embedding: dict):
    """
    위도, 경도 정보를 토대로 level-6 geohash로 변환한 뒤, 해당 임베딩을 반환
    """
    geo = geohash2.encode(lat, lon, precision=6)
    
    if geo in space_embedding:
        return torch.tensor(space_embedding[geo], dtype=torch.float)
    else:
        return torch.zeros(len(next(iter(space_embedding.values()))), dtype=torch.float)
    

# 각 임베딩 종합해 check-in 임베딩 만드는거 구현!

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255
0,dr5nzz,-0.005816,-0.061941,-0.007258,-0.149407,0.087255,0.049592,-0.026021,-0.011637,-0.071798,...,-0.040654,-0.078208,-0.003532,0.087537,0.007328,-0.096417,-0.077228,0.035259,0.019525,0.080821
1,dr5ppn,0.057614,0.032389,-0.002620,-0.035605,0.018107,-0.015194,-0.065257,-0.077122,-0.159260,...,0.040310,-0.010801,-0.027393,0.008184,0.100376,-0.087305,-0.097422,0.013745,-0.025334,0.083497
2,dr5ppt,-0.012948,-0.034116,-0.006110,0.042880,0.060608,0.017096,-0.003272,0.011354,-0.031993,...,-0.025050,0.010804,0.011813,-0.000931,0.135458,0.006723,-0.106512,-0.062076,0.011347,0.029076
3,dr5pqc,0.084327,-0.063180,0.034260,0.111373,0.031046,0.033870,0.000328,0.094078,0.018988,...,-0.070604,0.004208,-0.001845,-0.060310,0.122277,-0.035353,-0.048917,-0.079668,-0.024255,0.039467
4,dr5pqe,0.051590,0.029823,0.041519,0.044444,0.014957,0.023343,-0.116379,0.076519,0.061449,...,-0.052729,0.104373,-0.014984,-0.003087,0.000872,0.057076,0.110699,0.003993,0.020916,-0.025764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,dr788j,0.096508,0.053654,0.025964,-0.058183,0.049846,0.057342,0.031898,-0.080863,-0.107898,...,-0.031917,-0.052303,-0.030597,0.042484,0.035521,-0.039304,-0.020460,0.064762,-0.019193,0.030562
1119,dr788u,0.113608,0.015106,0.019439,-0.073436,0.075045,0.040605,0.038478,-0.045801,-0.107992,...,-0.039634,-0.063183,-0.041774,0.001392,0.054970,-0.039334,-0.021182,0.019622,0.008956,0.080966
1120,dr789p,-0.028568,-0.014564,0.005818,-0.038485,-0.051643,-0.027258,0.096167,0.020454,-0.014891,...,-0.104992,0.011143,0.054277,-0.046316,-0.090630,0.008820,0.008324,0.015272,-0.103928,-0.025870
1121,dr78c0,-0.028513,-0.014577,0.005819,-0.038452,-0.051622,-0.027268,0.096156,0.020505,-0.014822,...,-0.105017,0.011152,0.054290,-0.046343,-0.090649,0.008838,0.008340,0.015203,-0.103918,-0.025854
